In [2]:
%pylab inline
from scipy.integrate import odeint

Populating the interactive namespace from numpy and matplotlib


In [ ]:
# Pretendo hacer posible el uso de diferentes metodos para solucionar
# las ecuaciones diferenciales descritas por dY/dt = der_fun
# Como no todos los metodos requieren la misma informacion sobre los Ys
# y los ts ya existentes (por ejemplo multistep ones like the Adams-Bash one
# requieren de varios momentos anteriores)

def runge_kutta_4th(der_fun, constantes, Ys, dYsdt, ts = None, t = None, timestep = None):
    # Ys es un diccionario con claves los elementos del array ts
    # dYsdt es un diccionario con claves los elementos del array ts #Asumo, creo que correctamente, que el tiempo me determina todo el estado
    # t es el tiempo en el que se quiere calcular el valor de Y
    # timestep es el time step que utiliza el metodo, no estrictamente necesario
    ## si se cuadran las cosas en el llamado, pero creo que es util tenerlo
    ## en caso de que el metodo se base en uno, como los Runge-Kuttas
    
    #TODO validación de argumentos
    
    # Calculo de el valor de Y luego de un timestep
    Y_ant = Ys[-1]
    k1 = fun_der(Y_ant, None, constantes, dYsdt)
    k2 = fun_der(Y_ant + delta_t * (1./2 * k1), None, constantes, dYsdt)
    k3 = fun_der(Y_ant + delta_t * (1./2 * k2), None, constantes, dYsdt)
    k4 = fun_der(Y_ant + delta_t * k3, None, constantes, dYsdt)
    
    # Que quien se encargue de guardarlo sea alguien mas, pues, puede que
    # decida simplemente calcular el Y en un tiempo que ya existe, solo para
    # utilizarlo en algo mas, quiza para no obligar que se use solo un metodo
    # para la solucion de los sistemas de masas
    return Y_ant + delta_t*(K1/6. + K2/3. + K3/3. + K4/6., constantes)

In [ ]:
def fun_der_grav(Y, t, constantes, dYsdt = None): #Sin t. Me parece conveniente un array de constantes 
    
    #Asume Y = array([r1, r2, ..., v1, v2, ...]) con rs y vs array([c1, c2, c3])
    
    #TODO Checar longitudes de arrays de masa y Y
    
    # Devolver la derivada si ésta ya ha sido calculada. 
    ##IMPORTANTE:
    ## Asume que no
    ## ocurre recalculo de derivadas, porque asumo que la funcion derivada
    ## solo depende del Y y el t donde se calcula.
    
    if t in dYsdt:
        return dYsdt[0]
    
    masas = array(constantes[: -1]) #Ultimo es G
    G = constantes[-1]
    
    n = len(masas)
    dYdt = array([]) # Array solucion
    
    # Agregar velocidades a la solucion
    np.append(dYdt, [Y[n:]], axis = 0)
    
    # Agregar aceleraciones
    for i in range(n):
        # Concatena lo que esta a izquierda y derecha para masas y vectores posicion actuales
        masas_act = np.concatenate( (masas[: i], masas[i + 1 :]) , axis = 0) 
        rs_act = np.concatenate( (Y[: i], [i + 1 : n]) , axis = 0) - Ys[i] # Respecto al actual
        # Suma de aceleraciones debidas a cada masa
        np.append(dYdt, np.sum( G * masas_act * rs_act / np.sqrt(np.sum(rs_act**2, axis = 1))**3 ), axis = 0)

    # Agregar resultado al diccionario de derivadas guardadas.
    dYsdt[t] = dYdt #TODO estudiar si el diccionario se pasa completo
    
    return dYdt

In [ ]:
class Sistema_Masas:
    
    def __init__(self, masas, Y_0, fun_derivada, #Cambio a Y_0 para no forzar aun la convencion que usare de posiciones velocidades
                 t_max = 100., t_actual = 0., 
                 metodo = runge_kutta_4th, timestp = 0.1, err_rel_fpos = 0.000000001):
        
        #TODO checks
        # check masas positivas, vectores 3D y mismo tamano y misma cantidad que de masas, 
        # t_max positivo, timestp > 0,
        # actual_time > 0 y < a t_max redondeado + timestp, 
        # err_rel_fpos > 0
        
        # Guardar variables necesarias para enteder el sistema y la informacion
        ## guardada
        self.masas = np.array(masas) #any sequence type probably
        self.N = len(masas)
        self.Ys = {}
        self.dYsdt = {} #TODO, por si quiero implementar metodo multistep
        self.ts = array([])
        self.fun_derivada = fun_derivada
        self.metodo = self.metodo_procesado(metodo) #TODO Para guardar resultados en dYsdt
        
        self.t_max = t_max - t_max % timesp
        self.timestp = timestp
        self.err_rel_fpos = err_rel_fpos
        
        self.periodica = False
        self.periodo = -100.
        
        # Cuadrar vectores Y en diccionario con keys los tiempos
        # #posiciones_0 = np.array(Y_0[: n])
        # #velocidades_0 = np.array(velocidades_0[n :])
        
        ## Agregar info inicial
        np.append(self.ts, 0.)
        self.Ys[ts[0]] = Y_0
        
        ## Calculo del sistema en cada tiempo hasta self.t_max
        self.calcular_Ys( )
        
        # Cambiar estado actual
        self.estado_actual = self.cambiar_estado(ts[0])
    
    def aumentar_t_max(self, t_max):
        
        #TODO Check t_max, if less than current, raise Exc, if periodic, raise Exception (to alert)
        
        # Cambiar el tiempo maximo
        self.t_max = self.redondear_t(t_max)
        
        # Agregar Ys adicionales
        self.calcular_Ys(restart = False)
    
    def cambiar_timestep(self, timestp):
        
        #TODO Check timestp
        
        # Cambiar el time step
        self.timestp = timestp
        
        # Recalcular Ys
        self.calcular_Ys(restart = True)
    
    def cambiar_t_max_y_timestep(self, t_max, timestp):
        
        #TODO Check times
        
        # Cambiar timestp y t_max
        self.timestp = timestp
        self.t_max = self.redondear_t(t_max)
        
        # Recalcular Ys
        self.calcular_Ys(restart = True)
    
    def cambiar_estado(self, t):
        
        # Cambiar tiempo y Y actuales
        self.t_actual = self.redondear_t(t)
        self.Y_actual = Ys[self.t_actual]
        
    def info_actual(self):
        # Retorna el tiempo y el vector Y del estado actual#Careful with copies
        
        return self.t_actual, self.Y_actual
    
    def energia_mecanica(self): #TODO
        # Del estado actual
        pass
        
    def energia_potencial(self): #TODO
        # Del estado actual
        pass
        
    def energia_total(self): #TODO
        # Del estado actual
        pass
    
    def animacion(self, tiempo, fps = 1./timestep): # TODO. Pensar luego
        pass
        
    
    def calcular_Ys(self, restart = False):
        
        # No checa periodicidad
        
        # Si restart, reasigna los tiempos, los Ys, los dYsdt
        ## y empieza en t = 0
        if restart:
            self.ts = array([0.])
            self.Ys = {Ys[ts[0]]}
            self.dYsdt = {}
            self.cambiar_estado(ts[0])
        
        t = ts[-1]
        
        while t < self.t_max:
            #TODO calculate next Y
            Y_sig = self.metodo(self.fun_derivada
            #TODO if new Y is good enough to determine periodicity, change period stuff, t_max and break
            #TODO add new Y
            t += self.timestp
        
   # #def fuerza_sobre(self, i, t):#TODO #puede que lo quite, pues necesito asumir formato de Y
   # #    
   # #    # redondear el tiempo
   # #    return 0. #retornar fuerza, es las ultimas 3 coordenadas de la funcion derivada, multiplicada cada una por la masa correspondiente
    
    def redondear_tiempo(self, t): #Mejor llamarla en caso de querer utilizar un metodo que no utilice pasos uniformes
        # Asume que t es positivo
        
        if t > self.t_max + self.timestp:
            raise Exception('Tiempo muy largo, aumentar primero tiempo maximo')
        
        return t - t % timestp
                                
    def procesar_metodo(self, metodo): #TODO, 
        return metodo
    
    

# 3-Coreografía

## a.

In [ ]:
a = 0.24250109
b = −0.0607718825
c = 0.93240737
d = 0.86473146

In [3]:
ara = array([[1., 2., 3.], [4., 5., 6.]])

print np.sum(ara, axis = 0)
print np.sum(ara, axis = 1)
print np.sum(ara**2, axis = 1)

[ 5.  7.  9.]
[  6.  15.]
[ 14.  77.]
